In [3]:
# Let's use create a bot with strucutre

In [1]:
# same setup

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

SYS_MSG = "You are a synonym and antonym bot. Given a word, you will provide synonyms and antonyms for it."

In [2]:
# LCEL

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

prompt = ChatPromptTemplate.from_messages([("system", SYS_MSG), ("human", "{word}")])

chain = prompt | llm

In [3]:
chain.invoke({"word": "excited"})

AIMessage(content='Synonyms for "excited":\n1. thrilled\n2. exhilarated\n3. elated\n4. enthusiastic\n5. eager\n6. pumped\n7. animated\n8. electrified\n9. jubilant\n10. ecstatic\n\nAntonyms for "excited":\n1. bored\n2. calm\n3. indifferent\n4. uninterested\n5. apathetic\n6. disinterested\n7. relaxed\n8. composed\n9. tranquil\n10. cool', response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 36, 'total_tokens': 138}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 's

In [18]:
# this is difficult to work with, since it is an AI Message. Let's get to know output parsers that make our life easier.
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

chain.invoke({"word": "excited"})

'Synonyms for "excited": thrilled, exhilarated, enthusiastic, elated, animated, ecstatic, eager, pumped, fired up, electrified\n\nAntonyms for "excited": bored, uninterested, disinterested, indifferent, apathetic, calm, composed, relaxed, unenthusiastic, blasé'

In [20]:
# nice! but for building nice apps, we will need some structure

from typing import List

from langchain_core.output_parsers import JsonOutputParser

chain = prompt | llm | JsonOutputParser()

try:
    chain.invoke({"word": "excited"})
except:  # noqa: E722 (we know this is bad! :D)
    print("FAILURE! Why?")

FAILURE! Why?


In [21]:
# our model output is not json formatter. let's adjust the prmopt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_MSG + "\n. Make sure your output is a valid JSON structure."),
        ("human", "{word}"),
    ]
)

chain = prompt | llm | JsonOutputParser()

chain.invoke({"word": "excited"})

{'word': 'excited',
 'synonyms': ['thrilled', 'enthusiastic', 'eager', 'elated', 'animated'],
 'antonyms': ['calm', 'indifferent', 'apathetic', 'unenthusiastic', 'bored']}

In [32]:
# Nice! But, we can do better. OOP and all that.
from pydantic import BaseModel, Field

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

class SynonymAntonym(BaseModel):
    word: str = Field(
        description="The word for which synonyms and antonyms are provided"
    )
    synonyms: List[str] = Field(
        description="List of synonyms for the word, at least one synonym is required"
    )
    antonyms: List[str] = Field(description="List of antonyms for the word, if any")


parser = PydanticOutputParser(pydantic_object=SynonymAntonym)

# note that we also inject the instructions, so this prompt template is more future proof
prompt = PromptTemplate(
    template=SYS_MSG + "\n{format_instructions}",
    input_variables=["word"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = prompt | llm | parser

chain.invoke(
    {"word": "excited"}
)

SynonymAntonym(word='happy', synonyms=['joyful', 'content', 'delighted', 'pleased', 'glad'], antonyms=['sad', 'unhappy', 'miserable', 'depressed'])